In [1]:
import os
# import psycopg2
from dotenv import load_dotenv

from sqlalchemy import create_engine
from sqlalchemy.engine import URL

load_dotenv()

host = os.environ.get("DATABASE_HOST")
port = os.environ.get("DATABASE_PORT")
dbname = os.environ.get("DATABASE_DBNAME")
username = os.environ.get("DATABASE_USERNAME")
password = os.environ.get("DATABASE_PASSWORD")

url = URL.create(
    drivername="postgresql",
    username=username,
    host=host,
    database=dbname,
    password=password
)
# conn_string = f"host='{host}' dbname='{dbname}' user='{username}' password='{password}'"

def get_engine():
    return create_engine(url)

In [2]:
import pandas as pd
# import modules.db as db

def export_db(string):
    return pd.read_sql_table(string, get_engine(), index_col='id')

In [3]:
import pandas as pd
# import modules.pandas_tool as pt

table_name = 'dataset'
df = export_db(table_name)
df

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0


In [4]:
df.isnull().any(axis=1).sum()

201

In [5]:
df['bmi'].isnull().sum() / 5110

0.03933463796477495

In [6]:
#
df_fillna = df.copy()
df_fillna['bmi'].fillna(round(df['bmi'].mean(), 1), inplace=True)
df_fillna


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.9,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
18234,Female,80.0,1,0,Yes,Private,Urban,83.75,28.9,never smoked,0
44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0


In [19]:
df['ever_married'].unique(), df['work_type'].unique(), df['Residence_type'].unique(), df['smoking_status'].unique()

(array(['Yes', 'No'], dtype=object),
 array(['Private', 'Self-employed', 'Govt_job', 'children', 'Never_worked'],
       dtype=object),
 array(['Urban', 'Rural'], dtype=object),
 array(['formerly smoked', 'never smoked', 'smokes', 'Unknown'],
       dtype=object))

In [7]:
df_fillna.isnull().any(axis=1).sum()

0

In [8]:
df_fillna.duplicated().value_counts()

False    5110
dtype: int64

In [9]:
for col in ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']:
    df_fillna[col] = df_fillna[col].astype('category')

In [10]:
#
#상관계수를 알아보고 가설을 세우기 전에 일단 train, val, test 셋으로 나눕니다.
from sklearn.model_selection import train_test_split

df_nonTest, df_test = train_test_split(df_fillna, test_size= 0.2, random_state=42, stratify=df_fillna['stroke'])

In [11]:
df_nonTest.corr().sort_values('stroke')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_619032\595072433.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_nonTest.corr().sort_values('stroke')


,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
bmi,0.327496,0.159568,0.037480,0.164600,1.000000,0.039586
avg_glucose_level,0.232611,0.171633,0.166934,1.000000,0.164600,0.125328
heart_disease,0.267578,0.093292,1.000000,0.166934,0.037480,0.126883
hypertension,0.270335,1.000000,0.093292,0.171633,0.159568,0.133109
age,1.000000,0.270335,0.267578,0.232611,0.327496,0.243404
stroke,0.243404,0.133109,0.126883,0.125328,0.039586,1.000000


In [12]:
target = 'stroke'

X_test = df_test.drop(columns=target)
y_test = df_test[target]
X_nonTest = df_nonTest.drop(columns=target)
y_nonTest = df_nonTest[target]

In [13]:
#
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.utils import class_weight
import xgboost as xgb
import numpy as np
import gc
gc.enable()


model_xgb =  xgb.XGBClassifier(max_depth=5,
                              colsample_bytree=0.7,
                              n_estimators=2000,
                              learning_rate=0.2,
                              objective='binary:logistic', 
                              verbosity =1,
                              eval_metric  = 'auc',
                              tree_method='gpu_hist',
                              n_jobs=3,
                              enable_categorical=True,
                              early_stopping_rounds= 100                              
                              )


kfold = 10

folds = StratifiedKFold(n_splits=kfold)

val = np.zeros(X_nonTest.shape[0])
models = []
for fold_index, (train_index, val_index) in enumerate(folds.split(X_nonTest,y_nonTest)):
    print('Batch {} started...'.format(fold_index))
    gc.collect()

    classes_weights = class_weight.compute_sample_weight(
        class_weight='balanced',
        y=y_nonTest.iloc[train_index]
    )

    bst = model_xgb.fit(X_nonTest.iloc[train_index],y_nonTest.iloc[train_index],
              eval_set = [(X_nonTest.iloc[val_index],y_nonTest.iloc[val_index])],
              verbose= 200,
              sample_weight= classes_weights
              )
    models.append(model_xgb)
    # val[val_index] = model_xgb.predict_proba(X_nonTest.iloc[val_index])[:,1]
    # print('auc of this val set is {}'.format(roc_auc_score(y_nonTest.iloc[val_index],val[val_index])))
    ## pred += model_xgb.predict_proba(df_test.drop(columns=target))[:,1]/folds.n_splits

Batch 0 started...
[0]	validation_0-auc:0.67719
[107]	validation_0-auc:0.75630
Batch 1 started...
[0]	validation_0-auc:0.73483
[123]	validation_0-auc:0.80039
Batch 2 started...
[0]	validation_0-auc:0.70045
[106]	validation_0-auc:0.80848
Batch 3 started...
[0]	validation_0-auc:0.79377
[104]	validation_0-auc:0.86041
Batch 4 started...
[0]	validation_0-auc:0.74653
[103]	validation_0-auc:0.82596
Batch 5 started...
[0]	validation_0-auc:0.58271
[107]	validation_0-auc:0.79576
Batch 6 started...
[0]	validation_0-auc:0.72089
[155]	validation_0-auc:0.82584
Batch 7 started...
[0]	validation_0-auc:0.70231
[118]	validation_0-auc:0.85617
Batch 8 started...
[0]	validation_0-auc:0.67508
[107]	validation_0-auc:0.77892
Batch 9 started...
[0]	validation_0-auc:0.68782
[102]	validation_0-auc:0.78389


In [14]:
from sklearn.metrics import classification_report

pred = np.zeros(df_test.shape[0])
for model in models:
    pred += model.predict_proba(df_test.drop(columns=target))[:,1]/folds.n_splits
df_pred = pd.DataFrame(pred, columns = ['stroke'])
print(classification_report(pred > 0.5, df_test[target]))
print(roc_auc_score(pred > 0.5, df_test[target]))

              precision    recall  f1-score   support

       False       0.74      0.98      0.84       727
        True       0.78      0.13      0.23       295

    accuracy                           0.74      1022
   macro avg       0.76      0.56      0.53      1022
weighted avg       0.75      0.74      0.66      1022

0.5585363579138787


In [15]:
from sklearn.preprocessing import OrdinalEncoder
import eli5
from eli5.sklearn import PermutationImportance

feature_names = X_test.columns.tolist()
X_testEncoded = OrdinalEncoder().fit_transform(X_test)

importance_array=np.zeros(len(models))
for model in models:
    # permuter 정의
    permuter = PermutationImportance(
        estimator= model, # model
        scoring='roc_auc', # metric
        n_iter=5, # 다른 random seed를 사용하여 5번 반복
        random_state=42
    )
    permuter.fit(X_testEncoded, y_test)
    importance_array += permuter.feature_importances_ / len(models)

pd.Series(importance_array, feature_names).sort_values()



smoking_status      -0.000113
Residence_type      -0.000099
gender               0.000000
work_type            0.000652
hypertension         0.002866
bmi                  0.007595
ever_married         0.013844
heart_disease        0.019800
avg_glucose_level    0.020755
age                  0.152710
dtype: float64